In [27]:
import pandas as pd

In [28]:
data = pd.read_csv("/Users/kilianlorenz/Uni/Big_Data/Data/2019-Dec.csv") #Change Filepath

In [29]:

# Anzahl der Interaktionen pro Benutzer und Produkt zählen
interaction_counts = data.groupby(['user_id', 'product_id']).size().reset_index(name='interaction_count')


In [30]:

# Anzahl der Käufe pro Benutzer und Produkt zählen
purchase_counts = data[data['event_type'] == 'purchase'].groupby(['user_id', 'product_id']).size().reset_index(name='purchase_count')



In [31]:
# Zusammenführen der Interaktions- und Kaufzählungen
combined_data = pd.merge(interaction_counts, purchase_counts, on=['user_id', 'product_id'], how='left')



In [32]:
# Ersetzen von NaN-Werten mit 0, da keine Käufe gleichbedeutend mit 0 Käufen sind
combined_data['purchase_count'].fillna(0, inplace=True)



In [33]:
# Berechnung der Korrelation
correlation = combined_data['interaction_count'].corr(combined_data['purchase_count'])



In [34]:
print(f"Korrelation zwischen Interaktionsanzahl und Kaufanzahl: {correlation}")

Korrelation zwischen Interaktionsanzahl und Kaufanzahl: 0.5607976812408002
